## 9 — Mini-project (exercise)
1. Use the provided `df`.  
2. Create a cleaned version `df_clean` that:
   - Ensures `age` is numeric and reasonable (0 < age <= 110), fill missing with median.  
   - Coerces `income` to numeric and caps at 99th percentile.  
   - Parses `join_date` and keeps only year and month as `YYYY-MM` string.  
   - Keeps only rows with valid emails.  
3. Produce summary counts: rows before/after cleaning, number of imputed ages, number of capped incomes.

In [4]:
import pandas as pd
import numpy as np

np.random.seed(42)

data = {
    "id": range(1, 21),
    "age": [25, 30, None, 22, 120, 28, 27, None, 35, 29, 31, 27, 26, 24, None, 33, 40, -1, 29, 28],
    "income": [50000, 62000, 58000, None, 1_000_000, 54000, 52000, 51000, 60000, 59000,
               "65000", "not available", 48000, 47000, 45000, 70000, 72000, 68000, 66000, None],
    "join_date": ["2021-01-15", "2020/12/01", "15-02-2021", None, "2025-01-01",
                  "2021-06-03", "2021-07-15", "2021-07-20", "2021-08-01", "2021-09-10",
                  "2021-10-05", "2021-11-11", "2021-12-01", "2022-01-02", "2022-02-03", None, "2020-05-05", "2019-03-03", "2021-04-04", "2021-05-05"],
    "email": ["ALICE@example.COM", "bob@Example.com", " carol@example.com ", "dave@example", None,
              "emma@example.com", "frank@example.COM", "grace@Example.COM", "heidi@ex.com", "ivan@example.com",
              "judy@example.com", "kate@example.com", "leo@example.com", "mia@example.com", "nick@example.com",
              "oliver@example.com", "pam@example.com", "quinn@example.com", "ruth@example.com", "sam@example.com"],
    "notes": ["good", "", "n/a", None, "VIP customer", "follow up", "NA", "NA", "ok", "ok", "ok", "n/a", "", "good", None, "", "good", "ok", "n/a", "follow up"]
}

df = pd.DataFrame(data)
df.head(10)

,id,age,income,join_date,email,notes
0,1,25.0,50000,2021-01-15,ALICE@example.COM,good
1,2,30.0,62000,2020/12/01,bob@Example.com,
2,3,NaN,58000,15-02-2021,carol@example.com,n/a
3,4,22.0,None,None,dave@example,None
4,5,120.0,1000000,2025-01-01,None,VIP customer
5,6,28.0,54000,2021-06-03,emma@example.com,follow up
6,7,27.0,52000,2021-07-15,frank@example.COM,NA
7,8,NaN,51000,2021-07-20,grace@Example.COM,NA
8,9,35.0,60000,2021-08-01,heidi@ex.com,ok
9,10,29.0,59000,2021-09-10,ivan@example.com,ok


In [5]:
# Mini-project Solution
df_clean = df.copy()
rows_before = len(df_clean)

# age: numeric, 0-110, fill with median
df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')
df_clean.loc[(df_clean['age'] <= 0) | (df_clean['age'] > 110), 'age'] = np.nan
ages_to_impute = df_clean['age'].isna().sum()
age_median = df_clean['age'].median()
df_clean['age'] = df_clean['age'].fillna(age_median)

# income: numeric, cap at 99th percentile
df_clean['income'] = pd.to_numeric(df_clean['income'], errors='coerce')
p99 = df_clean['income'].quantile(0.99)
incomes_capped = (df_clean['income'] > p99).sum()
df_clean['income'] = df_clean['income'].clip(upper=p99)

# join_date: parse to YYYY-MM
df_clean['join_date_parsed'] = pd.to_datetime(df_clean['join_date'], errors='coerce', infer_datetime_format=True)
df_clean['join_ym'] = df_clean['join_date_parsed'].dt.to_period('M').astype(str).fillna('missing')

# email: keep only valid emails
df_clean['email_clean'] = df_clean['email'].str.strip().str.lower()
valid_mask = df_clean['email_clean'].str.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', na=False)
df_clean = df_clean[valid_mask].reset_index(drop=True)

rows_after = len(df_clean)
print(f"Before: {rows_before}, After (valid emails): {rows_after}")
print(f"Imputed ages: {ages_to_impute}, Income capped: {incomes_capped}, p99: ${p99:,.0f}")
df_clean[['id', 'age', 'income', 'join_ym', 'email_clean']].head(18)

Before: 20, After (valid emails): 18
Imputed ages: 5, Income capped: 1, p99: $851,520


/var/folders/4z/kp7dhv_100x6f98d2f4gzjzr0000gn/T/ipykernel_5308/43990825.py:19: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_clean['join_date_parsed'] = pd.to_datetime(df_clean['join_date'], errors='coerce', infer_datetime_format=True)


,id,age,income,join_ym,email_clean
0,1,25.0,50000.0,2021-01,alice@example.com
1,2,30.0,62000.0,NaT,bob@example.com
2,3,28.0,58000.0,NaT,carol@example.com
3,6,28.0,54000.0,2021-06,emma@example.com
4,7,27.0,52000.0,2021-07,frank@example.com
5,8,28.0,51000.0,2021-07,grace@example.com
6,9,35.0,60000.0,2021-08,heidi@ex.com
7,10,29.0,59000.0,2021-09,ivan@example.com
8,11,31.0,65000.0,2021-10,judy@example.com
9,12,27.0,NaN,2021-11,kate@example.com
